## Serving Tensorflow

TensorFlow actually has a built-in method to serve models, which is already optimized for use in production. It's also packaged as something that can easily be deployed on top of OpenShift

In this notebook, we'll set it up, and use a client to access it

First, we can log into the OpenShift cluster we're running on

In [ ]:
!oc login "${OC_API_URL}" --username="${OC_USER}" \
               --password="${OC_PASSWORD}" --insecure-skip-tls-verify='true'
!oc new-project "summit-user-${JUPYTERHUB_USER}"

Now we can create a new OpenShift app. TensorFlow contains support for serving a model stored in S3, so we can take advantage of that to serve the model from

In [ ]:
!oc new-app --name="tf-serving" \
             --docker-image="tensorflow/serving" \
             -e MODEL_BASE_PATH="s3://${JUPYTERHUB_USER}" \
             -e MODEL_NAME="mnist-model" \
             -e AWS_LOG_LEVEL=3 \
             -e AWS_ACCESS_KEY_ID="${AWS_ACCESS_KEY_ID}" \
             -e AWS_SECRET_ACCESS_KEY="${AWS_SECRET_ACCESS_KEY}" \
             -e AWS_REGION="us-east-1" \
             -e S3_ENDPOINT="$(echo ${S3_ENDPOINT} | sed 's%http://%%')" \
             -e S3_USE_HTTPS=0 \
             -e S3_VERIFY_SSL=0

The port needs to be exposed, since it exposes multiple ports, we'll redirect 8501 (the JSON api) to 80 for ease of use. This will allow us to verify it works via the browser.

In [ ]:
!oc expose svc/tf-serving --port 8501

Using `oc status` will let us see the URL that is exposed. Navigating to the url in the browser should return a 404, so instead navigate to `/v1/models/mnist-model`. It should return some metadata about the service

In [ ]:
!oc status | grep tf-serving

We can also double check it by curling the internal url or hostname, for which we'll want to use the internal port 8501

In [ ]:
!curl "$(oc get services | grep 'tf-serving' | awk '{print $3}'):8501/v1/models/mnist-model"

### Testing the model with a Flask App

To show some interactive testing, we have developed a small webapp that allows a user to draw a digit by hand and submit it to the model for classification.

The model takes in the image, greyscales it, pads it, normalizes it, etc, and transforms it into the format that the model expects, then POSTs it to the model's API

In [ ]:
!oc new-app --name "mnist-flask" "quay.io/llasmith/summit-workshop-flask-mnist:latest"
!oc expose svc/mnist-flask

To really experience the model, navigate to the URL that we exposed in the browser

Note: The app may take a few moments to spin up

In [ ]:
!oc status | grep mnist